In [ ]:
from word2vectraining import *
import csv
import random as rd
vocab_size = 23940
dim = 200
no_ofppl = 20
input_sentsize = 30
input_size_bot = dim*input_sentsize + no_ofppl

In [ ]:
#word2vec

def word2vec(vocab_size = vocab_size,h_size = dim):
    model = ke.Sequential()
    model.add(ke.layers.Dense(h_size, input_shape = (vocab_size,)))
    model.add(ke.layers.Dense(vocab_size,activation = ke.activations.softmax))
    return model

In [ ]:
#model creation
word2vec_model = word2vec()
word2vec_model.compile(loss="categorical_crossentropy",optimizer='adam', metrics=['accuracy'])

In [ ]:
#useful functions

def shuffle_csv(path,target_path):
    file = open(path,'r')
    csvreader = csv.reader(file)
    header = next(csvreader)
    rows_list = []
    for row in csvreader:
        rows_list.append(row)
    rd.shuffle(rows_list)
    outfile = open(target_path,'w')
    csvwriter = csv.writer(outfile)
    csvwriter.writerow(header)
    csvwriter.writerows(rows_list)
    file.close()

def fetch_rows(start,end,csv_path): #[) kinda interval
    file = open(csv_path,'r')
    csvreader = csv.reader(file)
    header = next(csvreader)
    tup_lst = []
    pos = 0
    for row in csvreader:
        if pos >= start and pos < end:
            tup = (eval(row[1]),eval(row[2]))
            tup_lst.append(tup)
        pos += 1
    file.close()
    return tup_lst

def fetch_next_batch(prev_end,batch_size,csv_path):
    batch = fetch_rows(prev_end,prev_end+batch_size,csv_path)
    if len(batch) != batch_size:
        return None
    else:
        return batch

In [ ]:
# training loop

no_of_epochs = 1
batch_size = 100
data_total_size = 307895

for epoch in range(1,no_of_epochs+1):
    print("Epoch: ",epoch)
    shuffle_csv("tupledata.csv","shuffled_tupledata.csv")
    prev_end = 0
    while batch != None:
        batch = fetch_next_batch(prev_end,batch_size,"shuffled_tupledata.csv")
        train_on_tup_batch(batch,word2vec_model)
        prev_end = prev_end + batch_size
